In [ ]:
#@title 🌐 🐍 Imports and Installs
!pip install uv
!uv pip install git+https://github.com/eigenP/utils.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.data import cells3d
from eigenp_utils.plotting_utils import hist_imshow, color_coded_projection
from eigenp_utils.intensity_rescaling import adjust_brightness_per_slice

In [ ]:
# Load 3D data (Nuclei channel)
# Shape: (Z, C, Y, X) -> we take channel 1 (Nuclei) -> (Z, Y, X)
image_raw = cells3d()[:, 1, ...]
print(f"Loaded image with shape: {image_raw.shape}")

In [ ]:
#@title ## Test `hist_imshow`
print("Testing hist_imshow...")
res = hist_imshow(image_raw)
res['fig'].show()

In [ ]:
#@title ## Test `color_coded_projection`
# Projects the Z-stack using color to represent depth (Z)
print("Testing color_coded_projection (Depth Coding)...")
# Input expects (Time, Y, X), so we pass our (Z, Y, X) directly
proj = color_coded_projection(image_raw, color_map='viridis')

plt.figure(figsize=(8, 8))
plt.imshow(proj)
plt.title("Depth-Coded Projection (Viridis)")
plt.axis('off')
plt.show()

### Test brightness correction by degrading the cells3d and then restoring it

In [ ]:
# Simulate signal decay along Z axis
print("Simulating signal decay...")
z_slices = image_raw.shape[0]

# Create exponential decay curve
decay_factor = np.exp(-np.linspace(0, 1.5, z_slices))

# Apply decay
image_degraded = image_raw * decay_factor[:, None, None]

# Add some noise
rng = np.random.default_rng(42)
noise = rng.normal(0, 10, image_degraded.shape)
image_degraded = image_degraded + noise

# Clip to valid range and cast back to original dtype
image_degraded = np.clip(image_degraded, 0, 65535).astype(image_raw.dtype)

print("Displaying degraded image...")
res_degraded = hist_imshow(image_degraded)
res_degraded['fig'].show()

In [ ]:
# Restore signal with adjust_brightness_per_slice (Gain Method)
print("Restoring signal with adjust_brightness_per_slice using method='gain'...")

# 'gamma_fit_func'='exponential' fits the decay curve
# 'method'='gain' multiplies slices by the inverse of the decay (Gain), preserving linearity better than Gamma.
image_restored = adjust_brightness_per_slice(image_degraded, gamma_fit_func='exponential', method='gain')

print("Displaying restored image...")
res_restored = hist_imshow(image_restored)
res_restored['fig'].show()

In [ ]:
# Compare Z-intensity profiles
print("Plotting Z-intensity profiles...")

mean_raw = image_raw.mean(axis=(1, 2))
mean_degraded = image_degraded.mean(axis=(1, 2))
mean_restored = image_restored.mean(axis=(1, 2))

plt.figure(figsize=(10, 6))
plt.plot(mean_raw, label='Original', linewidth=2, alpha=0.7)
plt.plot(mean_degraded, label='Degraded (Simulated)', linestyle='--', linewidth=2)
plt.plot(mean_restored, label='Restored (Gain)', linestyle='-.', linewidth=2)

plt.legend()
plt.title('Z-Axis Mean Intensity Profile')
plt.xlabel('Z Slice')
plt.ylabel('Mean Intensity')
plt.grid(True, alpha=0.3)
plt.show()